### Import Packages

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import preprocessing, utils, losses, layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
import os
import yaml
import numpy as np
from pickle import dump, load
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Load Data & Process

In [2]:
def load_conversation_data(data_directory):
    fileList = os.listdir(data_directory + "/")
    questions = []
    answers = []
    for file in fileList:
        data = yaml.safe_load(open(data_directory + "/" + file, "rb"))
        conversation_list = data["conversations"]
        for conversation in conversation_list:
            for i in range(len(conversation) - 1):
                questions.append(conversation[i])
                answers.append(conversation[i +1])
    answers_index = []
    for i in range(len(answers)):
        answers_index.append(i)
    return questions, answers, np.array(answers_index)

# load data from all files
questions, answers, answers_index = load_conversation_data("./English")

In [3]:
def vectorize(data):
    if type(data) == str:
       data = [data]
    return tokenizer.texts_to_sequences(data)

def devectorize(data):
    return answers[np.argmax(data)]

# define and fit tokenizer on questions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions)

x = pad_sequences(vectorize(questions))
y = answers_index
sentence_maxlen = len(x[0])

### Machine Learning Algorithms

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

models = [
    DecisionTreeClassifier(random_state=0),
    LogisticRegression(random_state=0)
]

# Precision, Recall and F1 Score
from sklearn.metrics import recall_score, precision_score, f1_score
from sklearn.model_selection import train_test_split

#For Each Algorithm 
accuracyList = []
for model in models:
    modelName = model.__class__.__name__
    #Split Data 
    X_train, X_test, y_train, y_test = train_test_split(x, answers, test_size=0.33, random_state=0)
    #Train Algorithm
    model.fit(X_train, y_train)
    # Make Predictions
    y_pred_proba = model.predict_proba(X_test)
    y_pred = model.predict(X_test)
    accuracyList.append([modelName, precision_score(y_test, y_pred, average='macro'), recall_score(y_test, y_pred, average='macro'), f1_score(y_test, y_pred, average='macro')])

pd.DataFrame(accuracyList, columns=["Algorithm", "Precision", "Recall", "F1 Score"])

/home/sheikh/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sheikh/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sheikh/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver opt

,Algorithm,Precision,Recall,F1 Score
0,DecisionTreeClassifier,0.001678,0.004474,0.002386
1,LogisticRegression,0.000840,0.002519,0.001259


### Deep Learning Algorithms

In [5]:
# define and compile the model
model = Sequential()
model.add(Dense(512, input_shape=x.shape))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(len(y)))
model.add(Activation('softmax'))
model.summary()
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 869, 512)          15872     
_________________________________________________________________
activation (Activation)      (None, 869, 512)          0         
_________________________________________________________________
dropout (Dropout)            (None, 869, 512)          0         
_________________________________________________________________
dense_1 (Dense)              (None, 869, 512)          262656    
_________________________________________________________________
activation_1 (Activation)    (None, 869, 512)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 869, 512)          0         
_________________________________________________________________
dense_2 (Dense)              (None, 869, 869)          4

In [6]:
# fit the model
model.fit(x,
          y,
          batch_size=100,
          epochs=1000,
          verbose=1,
          validation_split=0.1)

Epoch 1/1000
8/8 [==============================] - 0s 39ms/step - loss: 44.9144 - accuracy: 0.0000e+00 - val_loss: 18.4284 - val_accuracy: 0.0000e+00
Epoch 2/1000
8/8 [==============================] - 0s 21ms/step - loss: 17.5057 - accuracy: 0.0026 - val_loss: 10.9318 - val_accuracy: 0.0000e+00
Epoch 3/1000
8/8 [==============================] - 0s 21ms/step - loss: 9.4142 - accuracy: 0.0077 - val_loss: 8.8963 - val_accuracy: 0.0000e+00
Epoch 4/1000
8/8 [==============================] - 0s 21ms/step - loss: 7.4335 - accuracy: 0.0128 - val_loss: 8.1068 - val_accuracy: 0.0000e+00
Epoch 5/1000
8/8 [==============================] - 0s 20ms/step - loss: 6.9073 - accuracy: 0.0166 - val_loss: 7.8754 - val_accuracy: 0.0000e+00
Epoch 6/1000
8/8 [==============================] - 0s 18ms/step - loss: 6.6950 - accuracy: 0.0166 - val_loss: 7.8135 - val_accuracy: 0.0000e+00
Epoch 7/1000
8/8 [==============================] - 0s 18ms/step - loss: 6.5616 - accuracy: 0.0281 - val_loss: 7.8675 - va

8/8 [==============================] - 0s 17ms/step - loss: 4.3924 - accuracy: 0.1739 - val_loss: 24.3428 - val_accuracy: 0.0000e+00
Epoch 53/1000
8/8 [==============================] - 0s 17ms/step - loss: 4.3848 - accuracy: 0.1714 - val_loss: 24.4411 - val_accuracy: 0.0000e+00
Epoch 54/1000
8/8 [==============================] - 0s 18ms/step - loss: 4.2535 - accuracy: 0.1777 - val_loss: 24.7643 - val_accuracy: 0.0000e+00
Epoch 55/1000
8/8 [==============================] - 0s 18ms/step - loss: 4.2949 - accuracy: 0.1829 - val_loss: 25.0997 - val_accuracy: 0.0000e+00
Epoch 56/1000
8/8 [==============================] - 0s 17ms/step - loss: 4.1944 - accuracy: 0.1918 - val_loss: 26.2339 - val_accuracy: 0.0000e+00
Epoch 57/1000
8/8 [==============================] - 0s 19ms/step - loss: 4.2052 - accuracy: 0.1982 - val_loss: 26.9703 - val_accuracy: 0.0000e+00
Epoch 58/1000
8/8 [==============================] - 0s 27ms/step - loss: 4.1483 - accuracy: 0.1957 - val_loss: 27.3312 - val_accura

Epoch 108/1000
8/8 [==============================] - 0s 18ms/step - loss: 3.6083 - accuracy: 0.2609 - val_loss: 36.4222 - val_accuracy: 0.0000e+00
Epoch 109/1000
8/8 [==============================] - 0s 18ms/step - loss: 3.5402 - accuracy: 0.2621 - val_loss: 35.7592 - val_accuracy: 0.0000e+00
Epoch 110/1000
8/8 [==============================] - 0s 17ms/step - loss: 3.5119 - accuracy: 0.2660 - val_loss: 36.0070 - val_accuracy: 0.0000e+00
Epoch 111/1000
8/8 [==============================] - 0s 17ms/step - loss: 3.5486 - accuracy: 0.2442 - val_loss: 36.6233 - val_accuracy: 0.0000e+00
Epoch 112/1000
8/8 [==============================] - 0s 17ms/step - loss: 3.5529 - accuracy: 0.2609 - val_loss: 36.8059 - val_accuracy: 0.0000e+00
Epoch 113/1000
8/8 [==============================] - 0s 17ms/step - loss: 3.5848 - accuracy: 0.2596 - val_loss: 36.8936 - val_accuracy: 0.0000e+00
Epoch 114/1000
8/8 [==============================] - 0s 18ms/step - loss: 3.5288 - accuracy: 0.2660 - val_loss:

8/8 [==============================] - 0s 19ms/step - loss: 3.1766 - accuracy: 0.3299 - val_loss: 44.1441 - val_accuracy: 0.0000e+00
Epoch 164/1000
8/8 [==============================] - 0s 18ms/step - loss: 3.2008 - accuracy: 0.3107 - val_loss: 43.7007 - val_accuracy: 0.0000e+00
Epoch 165/1000
8/8 [==============================] - 0s 18ms/step - loss: 3.1948 - accuracy: 0.3095 - val_loss: 44.0693 - val_accuracy: 0.0000e+00
Epoch 166/1000
8/8 [==============================] - 0s 18ms/step - loss: 3.1393 - accuracy: 0.3043 - val_loss: 44.2526 - val_accuracy: 0.0000e+00
Epoch 167/1000
8/8 [==============================] - 0s 18ms/step - loss: 3.0912 - accuracy: 0.3350 - val_loss: 44.2843 - val_accuracy: 0.0000e+00
Epoch 168/1000
8/8 [==============================] - 0s 18ms/step - loss: 3.1156 - accuracy: 0.3146 - val_loss: 44.7792 - val_accuracy: 0.0000e+00
Epoch 169/1000
8/8 [==============================] - 0s 18ms/step - loss: 3.1365 - accuracy: 0.3184 - val_loss: 45.3685 - val_

8/8 [==============================] - 0s 18ms/step - loss: 2.8811 - accuracy: 0.3465 - val_loss: 49.3829 - val_accuracy: 0.0000e+00
Epoch 219/1000
8/8 [==============================] - 0s 17ms/step - loss: 2.8470 - accuracy: 0.3453 - val_loss: 49.9600 - val_accuracy: 0.0000e+00
Epoch 220/1000
8/8 [==============================] - 0s 18ms/step - loss: 2.8482 - accuracy: 0.3568 - val_loss: 49.3510 - val_accuracy: 0.0000e+00
Epoch 221/1000
8/8 [==============================] - 0s 18ms/step - loss: 2.8799 - accuracy: 0.3529 - val_loss: 49.4490 - val_accuracy: 0.0000e+00
Epoch 222/1000
8/8 [==============================] - 0s 17ms/step - loss: 2.9097 - accuracy: 0.3453 - val_loss: 49.7349 - val_accuracy: 0.0000e+00
Epoch 223/1000
8/8 [==============================] - 0s 18ms/step - loss: 2.9110 - accuracy: 0.3402 - val_loss: 49.2043 - val_accuracy: 0.0000e+00
Epoch 224/1000
8/8 [==============================] - 0s 17ms/step - loss: 2.8427 - accuracy: 0.3606 - val_loss: 49.6246 - val_

...

8/8 [==============================] - 0s 19ms/step - loss: 1.6088 - accuracy: 0.5256 - val_loss: 69.5786 - val_accuracy: 0.0000e+00
Epoch 769/1000
8/8 [==============================] - 0s 19ms/step - loss: 1.6763 - accuracy: 0.5141 - val_loss: 70.7985 - val_accuracy: 0.0000e+00
Epoch 770/1000
8/8 [==============================] - 0s 18ms/step - loss: 1.6694 - accuracy: 0.5102 - val_loss: 70.1473 - val_accuracy: 0.0000e+00
Epoch 771/1000
8/8 [==============================] - 0s 18ms/step - loss: 1.7152 - accuracy: 0.5026 - val_loss: 71.3984 - val_accuracy: 0.0000e+00
Epoch 772/1000
8/8 [==============================] - 0s 19ms/step - loss: 1.6723 - accuracy: 0.5038 - val_loss: 70.5382 - val_accuracy: 0.0000e+00
Epoch 773/1000
8/8 [==============================] - 0s 18ms/step - loss: 1.6668 - accuracy: 0.5064 - val_loss: 70.7122 - val_accuracy: 0.0000e+00
Epoch 774/1000
8/8 [==============================] - 0s 18ms/step - loss: 1.6900 - accuracy: 0.5064 - val_loss: 69.6068 - val_

8/8 [==============================] - 0s 19ms/step - loss: 1.6656 - accuracy: 0.5166 - val_loss: 69.3997 - val_accuracy: 0.0000e+00
Epoch 824/1000
8/8 [==============================] - 0s 18ms/step - loss: 1.6002 - accuracy: 0.5384 - val_loss: 69.3542 - val_accuracy: 0.0000e+00
Epoch 825/1000
8/8 [==============================] - 0s 18ms/step - loss: 1.5598 - accuracy: 0.5448 - val_loss: 69.6920 - val_accuracy: 0.0000e+00
Epoch 826/1000
8/8 [==============================] - 0s 18ms/step - loss: 1.6009 - accuracy: 0.5269 - val_loss: 69.8582 - val_accuracy: 0.0000e+00
Epoch 827/1000
8/8 [==============================] - 0s 19ms/step - loss: 1.5738 - accuracy: 0.5294 - val_loss: 69.7827 - val_accuracy: 0.0000e+00
Epoch 828/1000
8/8 [==============================] - 0s 20ms/step - loss: 1.6580 - accuracy: 0.5294 - val_loss: 69.9783 - val_accuracy: 0.0000e+00
Epoch 829/1000
8/8 [==============================] - 0s 18ms/step - loss: 1.5979 - accuracy: 0.5153 - val_loss: 71.4653 - val_

8/8 [==============================] - 0s 18ms/step - loss: 1.5010 - accuracy: 0.5537 - val_loss: 72.4186 - val_accuracy: 0.0000e+00
Epoch 879/1000
8/8 [==============================] - 0s 18ms/step - loss: 1.5359 - accuracy: 0.5243 - val_loss: 73.3153 - val_accuracy: 0.0000e+00
Epoch 880/1000
8/8 [==============================] - 0s 18ms/step - loss: 1.5689 - accuracy: 0.5448 - val_loss: 72.6226 - val_accuracy: 0.0000e+00
Epoch 881/1000
8/8 [==============================] - 0s 18ms/step - loss: 1.5797 - accuracy: 0.5396 - val_loss: 73.7915 - val_accuracy: 0.0000e+00
Epoch 882/1000
8/8 [==============================] - 0s 18ms/step - loss: 1.5011 - accuracy: 0.5473 - val_loss: 74.1264 - val_accuracy: 0.0000e+00
Epoch 883/1000
8/8 [==============================] - 0s 18ms/step - loss: 2.2106 - accuracy: 0.5499 - val_loss: 73.2410 - val_accuracy: 0.0000e+00
Epoch 884/1000
8/8 [==============================] - 0s 18ms/step - loss: 1.5886 - accuracy: 0.5217 - val_loss: 72.1697 - val_

8/8 [==============================] - 0s 19ms/step - loss: 1.5558 - accuracy: 0.5256 - val_loss: 72.3476 - val_accuracy: 0.0000e+00
Epoch 934/1000
8/8 [==============================] - 0s 19ms/step - loss: 1.5103 - accuracy: 0.5575 - val_loss: 73.2336 - val_accuracy: 0.0000e+00
Epoch 935/1000
8/8 [==============================] - 0s 18ms/step - loss: 1.5757 - accuracy: 0.5217 - val_loss: 73.6941 - val_accuracy: 0.0000e+00
Epoch 936/1000
8/8 [==============================] - 0s 18ms/step - loss: 1.4934 - accuracy: 0.5332 - val_loss: 73.1144 - val_accuracy: 0.0000e+00
Epoch 937/1000
8/8 [==============================] - 0s 18ms/step - loss: 1.5331 - accuracy: 0.5499 - val_loss: 74.3434 - val_accuracy: 0.0000e+00
Epoch 938/1000
8/8 [==============================] - 0s 19ms/step - loss: 1.6165 - accuracy: 0.5281 - val_loss: 72.2285 - val_accuracy: 0.0000e+00
Epoch 939/1000
8/8 [==============================] - 0s 19ms/step - loss: 1.5662 - accuracy: 0.5460 - val_loss: 71.9880 - val_

8/8 [==============================] - 0s 19ms/step - loss: 1.5305 - accuracy: 0.5575 - val_loss: 73.0952 - val_accuracy: 0.0000e+00
Epoch 989/1000
8/8 [==============================] - 0s 18ms/step - loss: 1.5231 - accuracy: 0.5371 - val_loss: 71.3316 - val_accuracy: 0.0000e+00
Epoch 990/1000
8/8 [==============================] - 0s 19ms/step - loss: 1.5882 - accuracy: 0.5345 - val_loss: 71.7871 - val_accuracy: 0.0000e+00
Epoch 991/1000
8/8 [==============================] - 0s 19ms/step - loss: 1.5568 - accuracy: 0.5448 - val_loss: 70.6838 - val_accuracy: 0.0000e+00
Epoch 992/1000
8/8 [==============================] - 0s 18ms/step - loss: 1.5087 - accuracy: 0.5422 - val_loss: 72.0014 - val_accuracy: 0.0000e+00
Epoch 993/1000
8/8 [==============================] - 0s 18ms/step - loss: 1.4584 - accuracy: 0.5614 - val_loss: 72.3350 - val_accuracy: 0.0000e+00
Epoch 994/1000
8/8 [==============================] - 0s 20ms/step - loss: 1.5183 - accuracy: 0.5358 - val_loss: 73.0879 - val_

In [7]:
# save the trained model
model.save('model.h5')

# pickle the extra files
with open('extrafiles.pkl', 'wb') as f:
    dump([tokenizer, sentence_maxlen, answers], f)
print('Model and extra files saved successfully!')

Model and extra files saved successfully!


### Load the Saved Model and Test

In [8]:
# load the model
model = keras.models.load_model('model.h5')

# load the extra files
with open('extrafiles.pkl', 'rb') as f:
    tokenizer, sentence_maxlen, answers = load(f)

In [9]:
def get_response(query):
    pad_que = pad_sequences(vectorize(query), maxlen=30)
    return devectorize(model.predict(pad_que))

while True:
    input_query = input()
    if input_query == '':
        break
    else:
        print(' - ' + get_response(input_query))

Hi
 - Hello
How are you?
 - I am just an artificial intelligence.
Do you eat?
 - I'm a computer, I can't eat or drink.

